# Semi-Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train the Support Vector Machine classifier. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>


Script by: Luis Lizcano-Sandoval (2021)<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu
Updated: 08/05/2022

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds and land
3. Apply Depth-Invariant Index (generates band-ratios B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (SVM)
6. Get confusion matrices and accuracies
7. Export classification images to EE Assets (.tiff)
8. Save matrices in GDrive (.xlxs)

In [ ]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/drive


In [ ]:
## Authenticate your EE account
!earthengine authenticate

In [ ]:
## Install some libraries [may need to restart runtime after]:
!pip install xlsxwriter

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass_scripts.git

**Load required libraries:**

In [ ]:
import os, sys #Import os, sys again if runtime was restarted.
sys.path.insert(0,'/content/seagrass_scripts')
sys.path.append('/content/seagrass_scripts/bin/')

from google.colab import auth
auth.authenticate_user()

import google
SCOPES = ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/earthengine']
CREDENTIALS, project_id = google.auth.default(default_scopes=SCOPES)

import ee
ee.Initialize(CREDENTIALS, project='earth-engine-252816')

In [ ]:
## Verify you loaded the EE module correctly:
from process import start_processing
print('EE version: ',ee.__version__)

**Some metadata:**

In [ ]:
## Define the source of your image, if from your EE Assets or EE Collections:
# imageSource = 'assets'  ## Read BOA imagery from EE Assets
imageSource = 'ee'      ## Read BOA imagery from EE Collections

## Define the type of satellite imagery - uncomment as needed:
satellite = 'Sentinel2'
# satellite = 'Landsat8'
# satellite = 'Landsat7'
# satellite = 'Landsat5'

## Settings for running script:
regionName = 'Laguna Madre MX' ## Region to classify [match metadata from ´regions´ collection below]
boaFolder = 'FL_20' ## Name of folder with BOA images in assets.
exportFolder = '2020_pre' ## Name of EE folder to save the final output - must create it manually if not exist yet.
dataFolder = 'Ground-points-19' ## EE Folder with Ground-truth points
smoothStr = '_raw_' # Smooth classified pixels or not? options: '_smooth_' or '_raw_'

## More metadata for saving image properties:
nameCode = 'LMMX' ## Unique code names
regionCountry = 'Mexico'
state = 'Tamaulipas'

**Insert list of image IDs:**

In [ ]:
## Paste image ids. The script can work for images from different tiles.

## Sentinel 2
imageList = [
'20201023T170359_20201023T171205_T14RPN',
'20201107T170521_20201107T171231_T14RPN',
'20201112T170539_20201112T171320_T14RPN',
'20201003T170049_20201003T170531_T14RPN'
]

## Landsat list
# 2019
# imageList = [
# 'LE07_017040_20190223',
# 'LE07_017040_20191005']

# 2000
# imageList = [
# 'LE07_017040_20000914',
# 'LT05_017040_20000602']

# 1990
# imageList = [
# 'LT05_017041_19990904',
# 'LT05_017041_19991107',
# 'LT05_017041_19990107'
# ]

# for i in range(len(imageList)):
#     print(imageList[i])

**Import data:**

In [ ]:
#########################    IMPORT TRAINING DATA    #########################

# Sandy areas
sand_areas = ee.FeatureCollection("users/lizcanosandoval/ground-points/Sand")
print('Sand areas loaded:', sand_areas.get('system:id').getInfo())

# Ground-Points
groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/"+dataFolder)
#groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/Turkey_2020")
print('Ground-points loaded:', groundPoints.get('system:id').getInfo())

## Landmask
#land = ee.FeatureCollection("users/lizcanosandoval/Florida_10m") ##Created from NDWI using Sentinel-2 imagery
land = ee.ImageCollection("users/lizcanosandoval/Watermask_S2") ##Created from NDWI using Sentinel-2 imagery
print('Land mask loaded:', land.get('system:id').getInfo())

## Region polygons:
regions = ee.FeatureCollection("users/lizcanosandoval/Seagrass/Regions")
print('Regions loaded:', regions.get('system:id').getInfo())

print('*** Collections loaded correctly! ***')

**Start classification loop:**

In [ ]:
%%time
start_processing(imageSource,satellite,regionName,boaFolder,exportFolder,dataFolder,smoothStr,nameCode,
                 regionCountry,state,imageList,sand_areas,groundPoints,land,regions,0,1)